# Работа со строковыми значениями

In [ ]:
import pandas as pd
from random import sample
from bs4 import BeautifulSoup
import re
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
import nltk

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны.

In [ ]:
recipes = pd.read_csv('recipes_sample.csv', usecols = ['id', 'minutes'])
recipes2 = recipes.sample(5)
recipes2

,id,minutes
12076,145126,90
815,446605,30
7100,482562,5
9410,47167,18
28837,73376,20


In [ ]:
print("|{0:^11}| {1:^6}|\n|{2}|".format("id", "minutes","-"*20))
for index, row in recipes2.iterrows():
    print("|{0:^11}| {1:^7}|".format(row['id'], row['minutes']))

|    id     | minutes|
|--------------------|
|  145126   |   90   |
|  446605   |   30   |
|  482562   |    5   |
|   47167   |   18   |
|   73376   |   20   |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__).
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [ ]:
from bs4 import BeautifulSoup

with open("steps_sample.xml",
          "r",
          encoding="utf-8") as fp:
    xml = BeautifulSoup(fp)
rcp = xml.find_all('recipe')
steps_sample = {recipe.id.text: [step.text for step in recipe.steps if step!='\n'] for recipe in rcp}

def show_info(name, steps, minutes, author_id):
    string = f'"{name.title()}"\n\n'
    for i in range(len(steps)):
        stp = f'{i+1}. {steps[i].capitalize()}\n'
        string += stp
    string += f'{"-":->10}\n'
    string += f'Автор: {author_id}\n'
    string += f'Среднее время приготовления: {minutes} минут\n'
    return string

def get_info(id):
    row = recipes.iloc[recipes[recipes.id == id].index[0]]

    name = row['name']
    steps = steps_sample[str(id)]
    minutes = row['minutes']
    author_id = row['contributor_id']
    return name, steps, minutes, author_id

info = get_info(170895)

print(show_info(info[0],info[1],info[2],info[3]))

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [ ]:
p = re.compile(r"[1-9][0-9]?\s(?:\bhours?\b|\bminutes?\b)")

for item in steps_sample['25082']:
    if p.findall(item) != []:
        print(f'{item}\n', p.findall(item))

turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board
 ['20 minutes']
when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly
 ['10 minutes']
let the dough rise until it springs back when you stick your finger in it , and it is about twice the size as it was before (this takes about 2 hours
 ['2 hours']
when the dough has risen twice , deflate it again and cover it with the damp cloth again for about 10 minutes , then divide it into and shape it into loaves , buns , etc
 ['10 minutes']
bake at 400 for 20 minutes , and then turn the oven down to 350 and bake for 20-30 minutes longer , until the loaf is a lovely brown and sounds hollow when you thump it on the bottom
 ['20 minutes', '30 minutes']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [ ]:
p1 = re.compile(r'^\bthis\b[\s\d\w]+,\s?\bbut\b', re.M)
rec = []
for item in recipes.description.dropna():
    if p1.findall(item) != []:
        rec.append(item)
print(f'Кол-во подходящих описаний - {len(rec)}')
sample(rec,3)

Кол-во подходящих описаний - 142


['this is really great to have on hand when you have little time, but want something fresh and hot for breakfast.',
 "this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.",
 'this spanish dish is a lot like our sloppy joes, but with a little bit more pizzazz.  i suggest serving this over rice.']

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [ ]:
p3 = re.compile(r" / ")
for item in steps_sample['72367']:
    print(p3.sub('/', item))

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [ ]:
recipes_dct = {}
for recipe in xml.find_all('recipe'):
    r_id = int(recipe.id.contents[0])
    steps = [step.text for step in recipe.find('steps').find_all('step')]
    recipes_dct[r_id] = steps

tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
uniq = []
#steps_sample['44123']

for id in steps_sample:
    for step in steps_sample[id]:
        row = tokenizer.tokenize(step)
        for word in row:
            uniq.append(word)

print('Кол-во уникальных слов - ', len(set(uniq)))

Кол-во уникальных слов -  15139


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [ ]:
recipess=pd.read_csv('recipes_sample.csv')
a = []
result_df = pd.DataFrame()

for i in range(recipess.shape[0]):
    try:
        recipe_id, length = recipess.iloc[i, 1], len(sent_tokenize(recipess.iloc[i, 6]))
    except TypeError:
        continue
    a.append((recipe_id, length))

a = sorted(a, key=lambda x: x[1], reverse=True)
id_list = [i for i, l in a[:5]]

for id in id_list:
    result_df = pd.concat([result_df, recipess[recipess.id == id]])

result_df

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
```
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [ ]:
w1 = 'I  omitted the raspberries and added strawberries instead'
w2 = recipes[recipes['id'] == 241106]['name'].values[0]

def pos_word(word):
    for i in pos_tag(word_tokenize(word)):
        print(f'{i[1]: ^{len(i[0])+len(i[1])}}', end = '')
    print('')
    for i in pos_tag(word_tokenize(word)):
        print(f'{i[0]: ^{len(i[0])+len(i[1])}}', end = '')
    print('\n')

pos_word(w1)
pos_word(w2)

PRP    VBD     DT       NNS       CC    VBD         NNS         RB    
 I   omitted   the  raspberries   and  added   strawberries   instead 

    JJ       NNS     IN      NNS       VBP     JJ    CC    JJ      NNS   
 eggplant  steaks   with  chickpeas   feta   cheese  and  black  olives  

